### **Summarization: An Unsupervised Extractive Way**

###### **There are two types of approach for text summarization: 1) Supervised and 2) Unsupervised. And in terms of summaries, there are two types of summaries: 1) Extractive and 2) Abstractive.<br>Here, we will use a Unsupervised extractive method.**

#### **Steps**

##### **1. Generate Embedding Model**
###### **Embedding selects sentences that contain words with the same meaning as the most relevant words — even if the words are different. So, before even preprocessing the text, the program must define the embedding model. To obtain the word embedding, I split the text into words and passed it to Word2vec.** 